<a href="https://colab.research.google.com/github/rahulrajpr/prepare-anytime/blob/main/spark/functions/13_spark_sql_aggregate_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Spark Aggregate Functions**
https://spark.apache.org/docs/latest/sql-ref-functions-builtin.html#aggregate-functions

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('spark-functions').getOrCreate()

In [ ]:
# count

sql = '''
with cte as
(
select sequence(1,100,2) as col
)
,cte2 as
(
select
   explode(col) as col
from cte
)
select count(col) as cnt
from cte2
'''
spark.sql(sql).show(truncate = False)

+---+
|cnt|
+---+
|50 |
+---+



In [ ]:
# count (distinct **)

sql = '''
with cte as
(
select array(1,1,1,1,1,2,2,22,3,3,3,10,100) as col
)
,cte2 as
(
select
   explode(col) as col
from cte
)
select count(distinct col) as cnt
from cte2
'''
spark.sql(sql).show(truncate = False)

+---+
|cnt|
+---+
|6  |
+---+



In [ ]:
# approx_count_distinct : approximate count distinct using the HyperLooLog++ function

data = [['rahul'],['rahul'],['skylr'],['lathika'],['hazel'],['jameela'],['lakshmi'],['lakshmi'],['skylr']]
schema = ['names']

dataframe = spark.createDataFrame(data, schema)
dataframe.selectExpr('approx_count_distinct(names) as appxcntDist').show(truncate = False)

+-----------+
|appxcntDist|
+-----------+
|6          |
+-----------+



###### COUNT vs. COUNT(DISTINCT) vs. APPROX_COUNT_DISTINCT

| Feature | `COUNT(column)` | `COUNT(DISTINCT column)` | `APPROX_COUNT_DISTINCT(column)` |
| :--- | :--- | :--- | :--- |
| **Primary Purpose** | Counts the total number of **non-null** values. | Calculates the **exact cardinality** (number of unique, non-null values). | **Estimates the cardinality** (number of unique, non-null values). |
| **What it Measures** | **Volume / Quantity** of data. | **Exact Uniqueness** (True Cardinality). | **Approximate Uniqueness** (Estimated Cardinality). |
| **Output** | Exact integer. | Exact integer. | Approximate integer (High accuracy, but not exact). |
| **Performance / Speed** | 🟢 **Very Fast**<br/>A simple scan and increment. | 🔴 **Slow / Expensive**<br/>Requires building a complete in-memory hash set of all unique values. | 🟡 **Very Fast**<br/>Uses a fixed-size probabilistic sketch (e.g., HyperLogLog). |
| **Accuracy** | 100% Accurate. | 100% Accurate. | **High, but not 100%.**<br/>Typical accuracy is 97-99.9% with a small, predictable error rate. |
| **Memory Usage** | Low (a single counter). | High (scales with the number of unique values). | Very Low (constant memory, fixed size of the sketch). |
| **Ideal Use Case** | "How many sales transactions were recorded?" | "How many unique customers placed an order?" (for financial reporting). | "How many unique visitors are on our site right now?" (for a real-time dashboard). |

In [ ]:
# count_min_sketch

data = [['rahul'],['rahul'],['skylr'],['lathika'],['hazel'],['jameela'],['lakshmi'],['lakshmi'],['skylr']]
schema = ['names']

dataframe = spark.createDataFrame(data, schema)
dataframe.selectExpr('count_min_sketch(names,CAST(0.1 AS DOUBLE),CAST(0.95 AS DOUBLE),42) as count_min_sketchOut').show(truncate = False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### Tabular Comparison: `approx_count_distinct` vs `count_min_sketch`

| Feature | `approx_count_distinct` | `count_min_sketch` |
|---------|------------------------|-------------------|
| **What it measures** | Number of unique values | Frequency of each value |
| **Output** | Single number | Probabilistic data structure |
| **Answers** | "How many distinct items?" | "How many times per item?" |
| **Use Case** | Unique visitor count | Visit frequency per user |
| **Example Result** | `5000` unique users | `user1: 15 visits`, `user2: 8 visits` |
| **Complexity** | Simple - ready to use | Complex - needs processing |
| **Practical Usage** | Direct results in PySpark | Hard to use in PySpark SQL |
| **Serialization** | No serialization needed | Returns serialized binary data |
| **Data Access** | Direct value | Requires deserialization to access counts |

**One-liner:** `approx_count_distinct` gives direct results, `count_min_sketch` gives serialized binary that's hard to use in PySpark.

In [ ]:
# count_if

sql = '''
with cte as
(
select sequence(1,100,2) as col
)
,cte2 as
(
select
   explode(col) as col
from cte
)
select count_if(col % 3 = 0) as cnt
from cte2
'''
spark.sql(sql).show(truncate=False)

+---+
|cnt|
+---+
|17 |
+---+



In [ ]:
# min

sql = '''
with cte as
(
select sequence(1,100,2) as col
)
,cte2 as
(
select
   explode(col) as col
from cte
)
select min(col) as val
from cte2
'''
spark.sql(sql).show(truncate=False)

+---+
|val|
+---+
|1  |
+---+



In [ ]:
# min_by

sql = '''
with cte as
(
select 1 as col1, 2 as col2
union all
select 2 as col1, 5 as col2
union all
select 3 as col1, 10 as col2
union all
select 4 as col1, -3 as col2
)
select
  min_by(col1,col2) as minOfCol1ByCol,
  min_by(col2,col1) as minOfCol2ByCol1
from cte
'''
spark.sql(sql).show(truncate = False)

#--

sql = '''
with cte as
(
select 1 as col1, 2 as col2
union all
select 1 as col1, -3 as col2
union all
select 3 as col1, 10 as col2
union all
select 4 as col1, -3 as col2
)
select
  min_by(col1,col2) as minOfCol1ByCol,
  min_by(col2,col1) as minOfCol2ByCol1
from cte
'''
spark.sql(sql).show(truncate = False)

+--------------+---------------+
|minOfCol1ByCol|minOfCol2ByCol1|
+--------------+---------------+
|4             |2              |
+--------------+---------------+

+--------------+---------------+
|minOfCol1ByCol|minOfCol2ByCol1|
+--------------+---------------+
|4             |-3             |
+--------------+---------------+



##### Key Point on `min_by`:
---
When there are duplicate minimum values in the ordering column, `min_by()` uses **first occurrence semantics** rather than returning all possible values or choosing randomly.
---

In [ ]:
# max

sql = '''
with cte as
(
select sequence(1,100,2) as col
)
,cte2 as
(
select
   explode(col) as col
from cte
)
select max(col) as val
from cte2
'''
spark.sql(sql).show(truncate=False)

+---+
|val|
+---+
|99 |
+---+



In [ ]:
# max_by

sql = '''
with cte as
(
select 1 as col1, 2 as col2
union all
select 2 as col1, 5 as col2
union all
select 3 as col1, 10 as col2
union all
select 4 as col1, -3 as col2
)
select
  max_by(col1,col2) as maxOfCol1ByCol,
  max_by(col2,col1) as maxOfCol2ByCol1
from cte
'''
spark.sql(sql).show(truncate = False)

#--

sql = '''
with cte as
(
select 1 as col1, 2 as col2
union all
select 1 as col1, -3 as col2
union all
select 3 as col1, 10 as col2
union all
select 4 as col1, -3 as col2
)
select
  max_by(col1,col2) as maxOfCol1ByCol,
  max_by(col2,col1) as maxOfCol2ByCol1
from cte
'''
spark.sql(sql).show(truncate = False)

+--------------+---------------+
|maxOfCol1ByCol|maxOfCol2ByCol1|
+--------------+---------------+
|3             |-3             |
+--------------+---------------+

+--------------+---------------+
|maxOfCol1ByCol|maxOfCol2ByCol1|
+--------------+---------------+
|3             |-3             |
+--------------+---------------+




##### Key Point:
---
When there are duplicate minimum values in the ordering column, `max_by()` uses **first occurrence semantics** rather than returning all possible values or choosing randomly.
---

In [ ]:
# sum

sql = '''
with cte as
(
select sequence(1,100,2) as col
)
,cte2 as
(
select
   explode(col) as col
from cte
)
select sum(col) as val
from cte2
'''
spark.sql(sql).show(truncate=False)

+----+
|val |
+----+
|2500|
+----+



In [ ]:
# try_sum

sql = '''
with cte as
(
select sequence(1,100,2) as col
)
,cte2 as
(
select
   explode(col) as col
from cte
)
select try_sum(col) as val
from cte2
'''
spark.sql(sql).show(truncate=False)

+----+
|val |
+----+
|2500|
+----+



In [ ]:
# avg

sql = '''
with cte as
(
select sequence(1,100,2) as col
)
,cte2 as
(
select
   explode(col) as col
from cte
)
select avg(col) as val
from cte2
'''
spark.sql(sql).show(truncate=False)

+----+
|val |
+----+
|50.0|
+----+



In [ ]:
# try_avg

sql = '''
with cte as
(
select sequence(1,100,2) as col
)
,cte2 as
(
select
   explode(col) as col
from cte
)
select try_avg(col) as val
from cte2
'''
spark.sql(sql).show(truncate=False)

+----+
|val |
+----+
|50.0|
+----+



In [ ]:
# mean

sql = '''
with cte as
(
select sequence(1,100,2) as col
)
,cte2 as
(
select
   explode(col) as col
from cte
)
select mean(col) as val
from cte2
'''
spark.sql(sql).show(truncate=False)

+----+
|val |
+----+
|50.0|
+----+



In [ ]:
# median

sql = '''
with cte as
(
select sequence(1,100,2) as col
)
,cte2 as
(
select
   explode(col) as col
from cte
)
select median(col) as val
from cte2
'''
spark.sql(sql).show(truncate=False)

+----+
|val |
+----+
|50.0|
+----+



In [ ]:
# mode

sql = '''
with cte as
(
select sequence(1,100,2) as col
)
,cte2 as
(
select
   explode(col) as col
from cte
)
select mode(col) as val
from cte2
'''
spark.sql(sql).show(truncate=False)

+---+
|val|
+---+
|13 |
+---+



---
#### AVG(MEAN) vs MEDIAN vs MODE
---
##### AVG / MEAN
- **Mathematical average**
- **Use when**: Data is normally distributed
- **Best for**: Symmetrical datasets without extreme values
---
##### MEDIAN  
- **Middle value** in sorted dataset
- **Use when**: Data has outliers or is skewed
- **Best for**: Income data, housing prices, skewed distributions
---
##### MODE
- **Most frequent value**
- **Use when**: Working with categorical data or finding popular items
- **Best for**: Survey responses, product preferences, categorical analysis
---
## Key Takeaway
- **Normal data** → Use AVG/MEAN
- **Skewed data with outliers** → Use MEDIAN  
- **Categorical data** → Use MODE
---

In [ ]:
# any : return true in case any value is true from the column

data = [[True],[True],[False],[False]]
schema = ['col1']
dataframe = spark.createDataFrame(data, schema)
dataframe.selectExpr('any(col1) as anyValue').show(truncate = False)

+--------+
|anyValue|
+--------+
|true    |
+--------+



In [ ]:
# some : return True in case any value is True in the column

data = [[True],[True],[False],[False]]
schema = ['col1']

dataframe = spark.createDataFrame(data, schema)
dataframe.selectExpr('some(col1) as someVaue').show(truncate = False)

+--------+
|someVaue|
+--------+
|true    |
+--------+



In [ ]:
# every : return True in case all value is True in the column

data = [[True],[True],[False],[False]]
schema = ['col1']

dataframe = spark.createDataFrame(data, schema)
dataframe.selectExpr('every(col1) as everyValue').show(truncate = False)

+----------+
|everyValue|
+----------+
|false     |
+----------+



In [ ]:
# bool_and : return true in case all values are true from the column

data = [[True],[True],[False],[False]]
schema = ['col1']
dataframe = spark.createDataFrame(data, schema)
dataframe.selectExpr('bool_and(col1) as bool_andValue').show(truncate = False)

+-------------+
|bool_andValue|
+-------------+
|false        |
+-------------+



In [ ]:
# bool_or : return true in case any value is true from the column

data = [[True],[True],[False],[False]]
schema = ['col1']
dataframe = spark.createDataFrame(data, schema)
dataframe.selectExpr('bool_or(col1) as bool_orValue').show(truncate = False)

+------------+
|bool_orValue|
+------------+
|true        |
+------------+



In [ ]:
# std : standard deviation

data_scores = [[85], [92], [78], [65], [88], [95], [72], [81]]
schema = ['score']

dataframe = spark.createDataFrame(data_scores, schema)
dataframe.selectExpr('std(score) as stdValue').show(truncate = False)

+----------------+
|stdValue        |
+----------------+
|10.1418510567422|
+----------------+



In [ ]:
# stddev : standard deviation

data_scores = [[85], [92], [78], [65], [88], [95], [72], [81]]
schema = ['score']

dataframe = spark.createDataFrame(data_scores, schema)
dataframe.selectExpr('std(score) as stdValue').show(truncate = False)

+----------------+
|stdValue        |
+----------------+
|10.1418510567422|
+----------------+



In [ ]:
# stddev_samp : standard deviation for sample

data_scores = [[85], [92], [78], [65], [88], [95], [72], [81]]
schema = ['score']

dataframe = spark.createDataFrame(data_scores, schema)
dataframe.selectExpr('stddev_samp(score) as stddev_sampValue').show(truncate = False)

+----------------+
|stddev_sampValue|
+----------------+
|10.1418510567422|
+----------------+



In [ ]:
# stddev_pop: standard deviation for population

data_scores = [[85], [92], [78], [65], [88], [95], [72], [81]]
schema = ['score']

dataframe = spark.createDataFrame(data_scores, schema)
dataframe.selectExpr('stddev_pop(score) as stddev_popValue').show(truncate = False)

+-----------------+
|stddev_popValue  |
+-----------------+
|9.486832980505138|
+-----------------+



In [ ]:
# variance: variance

data_scores = [[85], [92], [78], [65], [88], [95], [72], [81]]
schema = ['score']

dataframe = spark.createDataFrame(data_scores, schema)
dataframe.selectExpr('variance(score) as varianceValue').show(truncate = False)

+------------------+
|varianceValue     |
+------------------+
|102.85714285714286|
+------------------+



In [ ]:
# var_samp : variance for sample data

data_scores = [[85], [92], [78], [65], [88], [95], [72], [81]]
schema = ['score']

dataframe = spark.createDataFrame(data_scores, schema)
dataframe.selectExpr('var_samp(score) as var_sampValue').show(truncate = False)

+------------------+
|var_sampValue     |
+------------------+
|102.85714285714286|
+------------------+



In [ ]:
# var_pop : variance for population data

data_scores = [[85], [92], [78], [65], [88], [95], [72], [81]]
schema = ['score']

dataframe = spark.createDataFrame(data_scores, schema)
dataframe.selectExpr('var_pop(score) as var_popValue').show(truncate = False)

+------------+
|var_popValue|
+------------+
|90.0        |
+------------+



In [ ]:
# corr : correlation between two columns

data_scores = [[85,90], [92,10], [78,-30], [65,55], [88,20], [95,12], [72,2], [81,100]]
schema = ['col1','col2']

dataframe = spark.createDataFrame(data_scores, schema)
dataframe.selectExpr('corr(col1,col2) as corrValue').show(truncate = False)

+--------------------+
|corrValue           |
+--------------------+
|-0.09010476814935589|
+--------------------+



In [ ]:
# covar_samp : covariance with sample data

data_scores = [[85,90], [92,10], [78,-30], [65,55], [88,20], [95,12], [72,2], [81,100]]
schema = ['col1','col2']

dataframe = spark.createDataFrame(data_scores, schema)
dataframe.selectExpr('covar_samp(col1,col2) as covar_sampValue').show(truncate = False)

+-------------------+
|covar_sampValue    |
+-------------------+
|-41.285714285714285|
+-------------------+



In [ ]:
# covar_pop : covariance with population data

data_scores = [[85,90], [92,10], [78,-30], [65,55], [88,20], [95,12], [72,2], [81,100]]
schema = ['col1','col2']

dataframe = spark.createDataFrame(data_scores, schema)
dataframe.selectExpr('covar_pop(col1,col2) as covar_popValue').show(truncate = False)

+--------------+
|covar_popValue|
+--------------+
|-36.125       |
+--------------+



In [ ]:
# kurtosis : # kurtosis : variance for population data

data_scores = [[85], [92], [78], [65], [88], [95], [72], [81]]
schema = ['score']

dataframe = spark.createDataFrame(data_scores, schema)
dataframe.selectExpr('kurtosis(score) as kurtosisValue').show(truncate = False)

+-------------------+
|kurtosisValue      |
+-------------------+
|-0.9364814814814815|
+-------------------+



In [ ]:
# skewness : # kurtosis : variance for population data

data_scores = [[85], [92], [78], [65], [88], [95], [72], [81]]
schema = ['score']

dataframe = spark.createDataFrame(data_scores, schema)
dataframe.selectExpr('skewness(score) as skewnessValue').show(truncate = False)

+-------------------+
|skewnessValue      |
+-------------------+
|-0.3715676250697846|
+-------------------+



In [ ]:
# first :
# first_value

# the second argument stands for isIgnoreNull

data_scores = [[85,90], [92,10], [78,-30], [65,55], [88,20], [95,12], [72,2], [81,100]]
schema = ['col1','col2']

dataframe = spark.createDataFrame(data_scores, schema)
dataframe.selectExpr('first(col1,True) as firstValue').show(truncate = False)

+----------+
|firstValue|
+----------+
|85        |
+----------+



In [ ]:
# first_value

# the second argument stands for isIgnoreNull

data_scores = [[85,90], [92,10], [78,-30], [65,55], [88,20], [95,12], [72,2], [81,100]]
schema = ['col1','col2']

dataframe = spark.createDataFrame(data_scores, schema)
dataframe.selectExpr('first_value(col1,True) as first_valueValue').show(truncate = False)

+----------------+
|first_valueValue|
+----------------+
|85              |
+----------------+



In [ ]:
# last :
# synonym : last_value

# the second argument stands for isIgnoreNull

data_scores = [[85,90], [92,10], [78,-30], [65,55], [88,20], [95,12], [72,2], [81,100]]
schema = ['col1','col2']

dataframe = spark.createDataFrame(data_scores, schema)
dataframe.selectExpr('last(col1,True) as lastValue').show(truncate = False)

+---------+
|lastValue|
+---------+
|81       |
+---------+



In [ ]:
# last_value :

# the second argument stands for isIgnoreNull

data_scores = [[85,90], [92,10], [78,-30], [65,55], [88,20], [95,12], [72,2], [81,100]]
schema = ['col1','col2']

dataframe = spark.createDataFrame(data_scores, schema)
dataframe.selectExpr('last_value(col1,True) as last_valueValuue').show(truncate = False)

+----------------+
|last_valueValuue|
+----------------+
|81              |
+----------------+



In [ ]:
# any_value : a random value from a column

## second argument is for ignoring the NULLS

from pyspark.sql.types import IntegerType
from pyspark.sql.functions import cast,col

data_scores = [[85], [92], [78], [65], [88], [95], [72], [81], [None]]
schema = ['score']

dataframe = spark.createDataFrame(data_scores, schema).withColumn('score', col('score').cast('integer'))
dataframe.printSchema()
dataframe.selectExpr('any_value(score,True) as any_valueValue').show(truncate = False)

root
 |-- score: integer (nullable = true)

+--------------+
|any_valueValue|
+--------------+
|85            |
+--------------+



In [ ]:
# percentile : percentile for the given thresholds percentages

from pyspark.sql.types import IntegerType
from pyspark.sql.functions import cast,col

data_scores = [[85], [92], [78], [65], [88], [95], [72], [81], [None]]
schema = ['score']

dataframe = spark.createDataFrame(data_scores, schema).withColumn('score', col('score').cast('integer'))
dataframe.printSchema()
dataframe.selectExpr('percentile(score, array(0.25,0.5,0.75)) as percentileValue').show(truncate = False)

root
 |-- score: integer (nullable = true)

+------------------+
|percentileValue   |
+------------------+
|[76.5, 83.0, 89.0]|
+------------------+



In [ ]:
# percentile_approx : percentile for the given thresholds percentages (approximate))

# synonym : approx_percentile

from pyspark.sql.types import IntegerType
from pyspark.sql.functions import cast,col

data_scores = [[85], [92], [78], [65], [88], [95], [72], [81], [None]]
schema = ['score']

dataframe = spark.createDataFrame(data_scores, schema).withColumn('score', col('score').cast('integer'))
dataframe.printSchema()
dataframe.selectExpr('percentile_approx(score, array(0.25,0.5,0.75)) as percentile_approxValue').show(truncate = False)

root
 |-- score: integer (nullable = true)

+----------------------+
|percentile_approxValue|
+----------------------+
|[72, 81, 88]          |
+----------------------+



In [ ]:
# approx_percentile : percentile for the given thresholds percentages (approximate)

from pyspark.sql.types import IntegerType
from pyspark.sql.functions import cast,col

data_scores = [[85], [92], [78], [65], [88], [95], [72], [81], [None]]
schema = ['score']

dataframe = spark.createDataFrame(data_scores, schema).withColumn('score', col('score').cast('integer'))
dataframe.printSchema()
dataframe.selectExpr('approx_percentile(score, array(0.25,0.5,0.75)) as approx_percentileValue').show(truncate = False)

root
 |-- score: integer (nullable = true)

+----------------------+
|approx_percentileValue|
+----------------------+
|[72, 81, 88]          |
+----------------------+



In [ ]:
# collect_list : creating a list within a group
# it maintaines all the elements and with the order

from pyspark.sql.functions import collect_list

data = [
    ("Alice", "Math", 85),
    ("Alice", "Science", 92),
    ("Alice", "English", 85),  # Duplicate score 85
    ("Bob", "Math", 78),
    ("Bob", "Science", 78),    # Duplicate score 78
    ("Bob", "English", 90)
]
schema = ["name", "subject", "score"]

dataframe = spark.createDataFrame(data,schema)
dataframe.groupBy('name').agg(collect_list(col('score'))).show(truncate = False)

+-----+-------------------+
|name |collect_list(score)|
+-----+-------------------+
|Alice|[85, 92, 85]       |
|Bob  |[78, 78, 90]       |
+-----+-------------------+



In [ ]:
# collect_set : creating a set within a group
# it may not maintain the order, and it will contain only the unique values"

from pyspark.sql.functions import collect_set

data = [
    ("Alice", "Math", 85),
    ("Alice", "Science", 92),
    ("Alice", "English", 85),  # Duplicate score 85
    ("Bob", "Math", 78),
    ("Bob", "Science", 78),    # Duplicate score 78
    ("Bob", "English", 90)
]
schema = ["name", "subject", "score"]

dataframe = spark.createDataFrame(data,schema)
dataframe.groupBy('name').agg(collect_set(col('score'))).show(truncate = False)

+-----+------------------+
|name |collect_set(score)|
+-----+------------------+
|Alice|[85, 92]          |
|Bob  |[78, 90]          |
+-----+------------------+



In [ ]:
# roll up

data = [
    ("North", "Apple", 100),
    ("North", "Banana", 150),
    ("North", "Apple", 50),  # Another sale for North/Apple
    ("South", "Apple", 200),
    ("South", "Banana", 250),
    ("South", "Orange", 75),
    ("East", "Apple", 120),
    ("East", "Banana", 80)
]

columns = ["Region", "Product", "Sales"]
dataframe = spark.createDataFrame(data, columns)

#- group by without roll up

dataframe.createOrReplaceTempView('dataframe_view')
sql = '''
select region,product, sum(sales) as salesTotal
from dataframe_view
group by region,product
order by region,product
'''
spark.sql(sql).show(truncate = False)

#- group by WITH roll up

dataframe.createOrReplaceTempView('dataframe_view')
sql = '''
SELECT region, product, SUM(sales) as salesTotal
FROM dataframe_view
GROUP BY ROLLUP(region, product)
ORDER BY region NULLS LAST, product NULLS LAST
'''
spark.sql(sql).show(truncate = False)

+------+-------+----------+
|region|product|salesTotal|
+------+-------+----------+
|East  |Apple  |120       |
|East  |Banana |80        |
|North |Apple  |150       |
|North |Banana |150       |
|South |Apple  |200       |
|South |Banana |250       |
|South |Orange |75        |
+------+-------+----------+

+------+-------+----------+
|region|product|salesTotal|
+------+-------+----------+
|East  |Apple  |120       |
|East  |Banana |80        |
|East  |NULL   |200       |
|North |Apple  |150       |
|North |Banana |150       |
|North |NULL   |300       |
|South |Apple  |200       |
|South |Banana |250       |
|South |Orange |75        |
|South |NULL   |525       |
|NULL  |NULL   |1025      |
+------+-------+----------+



---
##### GROUP BY vs GROUP BY WITH ROLLUP
---
| Aspect | GROUP BY (Without ROLLUP) | GROUP BY WITH ROLLUP |
|--------|---------------------------|----------------------|
| **Purpose** | Aggregates data at specified grouping levels only | Creates subtotals and grand totals across hierarchy |
| **Output Rows** | One row per unique combination of grouping columns | Multiple rows: details + subtotals + grand total |
| **Hierarchy** | Flat structure - single level | Multi-level hierarchical structure |
| **NULL Handling** | NULL represents actual NULL values from data | NULL represents summary rows (all groups) |
| **Result Types** | Only detailed aggregated rows | Detailed rows + subtotal rows + grand total row |
| **Data Completeness** | Partial view - only the specified groupings | Complete view with hierarchical summaries |

In [ ]:
# grouping : to indicate the the subtotal row of the grouping heirarchy

# this is only valid when you use the rollups
# if rollups is there, then there is no meaning of grouping

data = [
    ("North", "Apple", 100),
    ("North", "Banana", 150),
    ("North", "Apple", 50),  # Another sale for North/Apple
    ("South", "Apple", 200),
    ("South", "Banana", 250),
    ("South", "Orange", 75),
    ("East", "Apple", 120),
    ("East", "Banana", 80)
]

columns = ["Region", "Product", "Sales"]
dataframe = spark.createDataFrame(data, columns)

dataframe.createOrReplaceTempView('dataframe_view')
sql = '''
SELECT
    region,
    product,
    SUM(sales) as salesTotal,
    GROUPING(region) as regionGrouping,
    GROUPING(product) as productGrouping
FROM dataframe_view
GROUP BY ROLLUP(region, product)
ORDER BY region NULLS LAST, product NULLS LAST
'''
spark.sql(sql).show(truncate = False)



+------+-------+----------+--------------+---------------+
|region|product|salesTotal|regionGrouping|productGrouping|
+------+-------+----------+--------------+---------------+
|East  |Apple  |120       |0             |0              |
|East  |Banana |80        |0             |0              |
|East  |NULL   |200       |0             |1              |
|North |Apple  |150       |0             |0              |
|North |Banana |150       |0             |0              |
|North |NULL   |300       |0             |1              |
|South |Apple  |200       |0             |0              |
|South |Banana |250       |0             |0              |
|South |Orange |75        |0             |0              |
|South |NULL   |525       |0             |1              |
|NULL  |NULL   |1025      |1             |1              |
+------+-------+----------+--------------+---------------+



In [ ]:
# grouping_id : to indicate the the subtotal row of the grouping heirarchy

# this is only valid when you use the rollups
# if rollups is there, then there is no meaning of grouping_id
# it represents the binary of the group id

data = [
    ("North", "Apple", 100),
    ("North", "Banana", 150),
    ("North", "Apple", 50),  # Another sale for North/Apple
    ("South", "Apple", 200),
    ("South", "Banana", 250),
    ("South", "Orange", 75),
    ("East", "Apple", 120),
    ("East", "Banana", 80)
]

columns = ["Region", "Product", "Sales"]
dataframe = spark.createDataFrame(data, columns)

dataframe.createOrReplaceTempView('dataframe_view')
sql = '''
SELECT
    region,
    product,
    SUM(sales) as salesTotal,
    GROUPING(region) as regionGrouping,
    GROUPING(product) as productGrouping,
    GROUPING_ID() AS groupingId
FROM dataframe_view
GROUP BY ROLLUP(region, product)
ORDER BY region NULLS LAST, product NULLS LAST
'''
spark.sql(sql).show(truncate = False)



+------+-------+----------+--------------+---------------+----------+
|region|product|salesTotal|regionGrouping|productGrouping|groupingId|
+------+-------+----------+--------------+---------------+----------+
|East  |Apple  |120       |0             |0              |0         |
|East  |Banana |80        |0             |0              |0         |
|East  |NULL   |200       |0             |1              |1         |
|North |Apple  |150       |0             |0              |0         |
|North |Banana |150       |0             |0              |0         |
|North |NULL   |300       |0             |1              |1         |
|South |Apple  |200       |0             |0              |0         |
|South |Banana |250       |0             |0              |0         |
|South |Orange |75        |0             |0              |0         |
|South |NULL   |525       |0             |1              |1         |
|NULL  |NULL   |1025      |1             |1              |3         |
+------+-------+----